## Timedelta

1. 세션의 지속 시간을 분으로 계산하고 가장 긴 지속시간을 출력하시오(반올림 후 총 분만 출력)
2. 가장 많이 머무른 Page를 찾고 그 페이지의 머문 평균 시간을 구하시오 (반올림 후 총 시간만 출력) 
3. 사용자들이 가장 활발히 활동하는 시간대(예: 새벽, 오전, 오후, 저녁)를 분석하세요. 이를 위해 하루를 4개의 시간대로 나누고 각 시간대별로 가장 많이 시작된 세션의 수를 계산하고, 그 중에 가장 많은 세션 수를 출력하시오
    - 새벽: 0시 부터 6시 전
    - 오전: 6시 부터 12시 전
    - 오후: 12 부터 18시 전
    - 저녁: 18시 부터 0시 전

4. user가 가장 많이 접속 했던 날짜를 출력하시오. (예, 2023-02-17)

In [90]:
import pandas as pd
df = pd.read_csv('data/website.csv')
#print(df.info())

# 1번
df['StartTime'] = pd.to_datetime(df['StartTime'])
df['EndTime'] = pd.to_datetime(df['EndTime'])
df['Duration'] = df['EndTime'] - df['StartTime']
df['Duration_m'] = df['Duration'].dt.seconds/60
print("세션의 평균 지속시간:" , round(df['Duration_m'].max()))

# 2번
print("가장 많이 머무른 페이지:",df['Page'].max())
df['Duration_h'] = df[df['Page'] == 'Page5']['Duration_m']/60
mean_d = df['Duration_h'].mean()
print("페이지에 머문 평균시간 :", round(mean_d))

# 3번
def time_binning(hour):
    if 0 <= hour < 6:
        return "새벽"
    elif 6 <= hour < 12:
        return "오전"
    elif 12 <= hour < 18:
        return "오후"
    else:
        return "저녁"
    
df['구분'] = df['StartTime'].dt.hour.apply(time_binning)
print("가장 많은 세션 수:",df['구분'].value_counts()[0])

# 4번 . user가 가장 많이 접속했던 날짜를 구하시오.
print("가장많이 접속했던 날짜:", df['StartTime'].dt.date.value_counts().index[0])

세션의 평균 지속시간: 300
가장 많이 머무른 페이지: Page5
페이지에 머문 평균시간 : 3
가장 많은 세션 수: 2447
가장많이 접속했던 날짜: 2023-03-28


### T1-30
- 12월 25일 결제 금액(price)은 12월 총 결제금액의 몇 %인가? (정수로 출력)

In [62]:
import pandas as pd
df = pd.read_csv("data/payment.csv")
#print(df.info())
#print(df.head())
df['date'] = pd.to_datetime(df['date'],format = '%Y%m%d')
sum_12 = df[df['date'].dt.month == 12]['price'].sum()
sum_1225 = df[(df['date'].dt.month == 12) & (df['date'].dt.day == 25)]['price'].sum()
print(round(sum_1225/sum_12*100))

26


### 시계열 데이터2
- 주어진 데이터에서 2022년 5월 주말과 평일의 sales컬럼 평균값 차이를 구하시오 (소수점 둘째자리까지 출력, 반올림)
- 데이터셋 : basic2.csv

In [130]:
# 데이터 불러오기 (datatime컬럼 지정)
df = pd.read_csv("data/basic2.csv", parse_dates=['Date'])
#print(df.info())
df['Date_num'] = df['Date'].dt.dayofweek
df = df[(df['Date'].dt.year == 2022) & (df['Date'].dt.month == 5)]
#print(df['Weekend'].value_counts())
df['Weekend'] = df['Date_num'].apply(lambda x: x>= 5)

weekend_sales = df[df['Weekend'] == True]['Sales'].mean()
weekday_sales = df[df['Weekend'] == False]['Sales'].mean()
print(round(weekend_sales - weekday_sales,2))

3010339.1


### 시계열 데이터3
- 주어진 데이터에서 2022년 월별 Sales 합계 중 가장 큰 금액과 2023년 월별 Sales 합계 중 가장 큰 금액의 차이를 절대값으로 구하시오. 
- 단 Events컬럼이 '1'인경우 80%의 Salse값만 반영함(최종값은 소수점 반올림 후 정수 출력)

In [156]:
df = pd.read_csv("data/basic2.csv", parse_dates=['Date'])
#print(df.head())
#print(df.info())

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month


def sales_transform(x):
    if x['Events'] == 1:
        x['Sales2'] = x['Sales']
    else:
        x['Sales2'] = x['Sales']*0.8
    return x
df = df.apply(lambda x:sales_transform(x), axis = 1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    730 non-null    datetime64[ns]
 1   Sales   730 non-null    int64         
 2   PV      730 non-null    int64         
 3   UV      684 non-null    float64       
 4   Events  730 non-null    int64         
 5   year    730 non-null    int64         
 6   month   730 non-null    int64         
 7   Sales2  730 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 45.8 KB
None


### 시계열 데이터 4
- 주어진 데이터(basic2.csv)에서 주 단위 Sales의 합계를 구하고, 가장 큰 값을 가진 주와 작은 값을 가진 주의 차이를 구하시오(절대값)

In [5]:
df = pd.read_csv("data/basic2.csv", parse_dates=['Date'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    730 non-null    datetime64[ns]
 1   Sales   730 non-null    int64         
 2   PV      730 non-null    int64         
 3   UV      684 non-null    float64       
 4   Events  730 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 28.6 KB
None


### 시계열 데이터 5
- 주어진 데이터(basic2.csv)에서 "pv"컬럼으로 1일 시차(lag)가 있는 새로운 컬럼을 만들고(예: 1월 2일에는 1월 1일 pv데이터를 넣고, 1월 3일에는 1월 2일 pv데이터를 넣음),
- 새로운 컬럼의 1월 1일은 다음날(1월 2일)데이터로 결측치를 채운 다음, 
- Events가 1이면서 Sales가 1000000이하인 조건에 맞는 새로운 컬럼 합을 구하시오

In [6]:
df = pd.read_csv("data/basic2.csv", parse_dates=['Date'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    730 non-null    datetime64[ns]
 1   Sales   730 non-null    int64         
 2   PV      730 non-null    int64         
 3   UV      684 non-null    float64       
 4   Events  730 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 28.6 KB
None
